In [1]:
# 2 weeks together
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2) #+7

write_folder="/home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) 

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]
recent_2_weeks

[datetime.date(2020, 4, 4), datetime.date(2020, 3, 28)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_2_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
list_1_after_201806=list_1_after_201806+list_2_after_201806
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-03-28/MediaStormMasterWeekly20200331-112513-883.txt',
 '/home/jian/BigLots/MediaStorm_2020-04-04/MediaStormMasterWeekly20200407-110922-206.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
        print(df_all_new_master.shape)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))


(101305, 3)
(231658, 3)
231658
231656


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

273


In [7]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Upda

In [8]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/BiWeekly_New_SignUps/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]
print("len(uploaded_new_seg)"+str(len(uploaded_new_seg)))
previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

len(uploaded_new_seg)84


In [9]:
# Exclude the quarterly regular segments


In [10]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index),file_uploaded)

2020-04-08 10:12:30.307645 231656 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:12:30.647982 231655 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:12:30.830599 231655 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:12:31.027768 231655 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:12:33.935798 231651 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded

2020-04-08 10:12:51.550432 231640 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:12:51.814857 231639 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_H_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:12:51.933540 231639 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:12:52.054526 231639 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:12:52.120776 231639 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Activ

2020-04-08 10:12:59.599850 231636 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:12:59.763642 231636 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:12:59.838437 231636 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_L_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:12:59.975258 231636 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:00.119533 231636 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_P_Above5

2020-04-08 10:13:13.609692 231628 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:13.848618 231628 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:14.273959 231628 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:14.441209 231628 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_Legacy_Active_M_S_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:16.733654 231626 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//C_all_quarterly_segments_RFM20Q1.csv
2

2020-04-08 10:13:25.251650 231618 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:25.326870 231618 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:25.413528 231618 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:25.492179 231618 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_H_P_Above50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:27.527066 231615 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18

2020-04-08 10:13:36.250804 231603 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:36.442103 231603 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:36.837261 231603 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:36.932598 231603 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Above50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:37.016149 231603 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_L_P_Belo

2020-04-08 10:13:43.772211 231602 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:43.957368 231602 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsHigh_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:44.348882 231602 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureHigh_RFM20Q1.csv
2020-04-08 10:13:44.444829 231602 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_P_Unknown_Unknown_CompetitorsLow_FurinitureLow_RFM20Q1.csv
2020-04-08 10:13:44.518147 231602 /home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18//T_SOTF_Active_M_S_Above50K_MultiFamilyOrCondo_

2020-04-08 10:13:51.701759 231599 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2018-12-29/NewReward_2018-12-29_P.csv
2020-04-08 10:13:52.039590 231599 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2018-12-29/NewReward_2018-12-29_S.csv
2020-04-08 10:13:52.242246 231598 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2018-12-29/NewReward_2018-12-29_T.csv
2020-04-08 10:13:53.379358 231598 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-01-26/NewReward_2019-01-26_P.csv
2020-04-08 10:13:53.602040 231598 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-01-26/NewReward_2019-01-26_S.csv
2020-04-08 10:13:53.757617 231598 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-01-26/NewReward_2019-01-26_T.csv
2020-04-08 10:13:54.828657 231598 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-02-23/NewReward_2019-02-23_P.csv
2020-04-08 10:13:55.075000 231598 /home/jian/celery/BiWeekly_New_Sign

2020-04-08 10:14:15.924996 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-11-30/NewReward_2019-11-30_T.csv
2020-04-08 10:14:16.612342 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-12-14/NewReward_2019-12-14_P.csv
2020-04-08 10:14:16.838139 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-12-14/NewReward_2019-12-14_S.csv
2020-04-08 10:14:17.542991 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2019-12-14/NewReward_2019-12-14_T.csv
2020-04-08 10:14:18.232754 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-11/NewReward_2020-01-11_P.csv
2020-04-08 10:14:18.545539 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-11/NewReward_2020-01-11_S.csv
2020-04-08 10:14:19.553076 231595 /home/jian/celery/BiWeekly_New_SignUps/monthly_update/output_2020-01-11/NewReward_2020-01-11_T.csv
2020-04-08 10:14:19.968561 231595 /home/jian/celery/BiWeekly_New_Sign

In [11]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

231591


In [12]:
zipmap = pd.ExcelFile('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx')
zipmap=zipmap.parse("unique_zips_full_footprint",dtype=str)
zipmap['zip_type']=zipmap['zip_type'].replace("zips_10","trans_S")

zipmap['zipcodegroup'] = zipmap['zip_type'].apply(lambda x: x.split("_")[1])
print(zipmap.shape)
print(zipmap['zip_cd'].nunique())

zipmap=zipmap.rename(columns={"zip_cd":"customer_zip_code"})

zipmap=zipmap[['customer_zip_code','zipcodegroup']]
zipmap.head(2)

(12980, 3)
12980


,customer_zip_code,zipcodegroup
0,75040,P
1,20743,P


In [13]:
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())

df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment_new_signups'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots /uploads/big_lots_onboarding

['P' 'S' nan]
['P' 'S' 'T']
P 71200
S 24785
T 135606


In [14]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,71200
1,S,24785
2,T,135606


In [15]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment_new_signups
3,8d1b851bfee8225444058f02a9b743afb80892973c5b7a...,522b700ccb1e4985246e7d7abd89f5537fbf1b108fdb91...,NaN,NewReward_040420_T
5,3588e14969934b7b65cd5c881eb289dbb1ea801bd345bc...,317106c422ade042676642e430e89bbf05aa9e463f93e6...,63877,NewReward_040420_T


In [16]:
print(datetime.datetime.now())

2020-04-08 10:14:30.360309


In [17]:
# to do below 
print(datetime.datetime.now())

2020-04-08 10:14:30.366330


In [18]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
transport.close()
print("Done: ",datetime.datetime.now())

Done:  2020-04-08 10:21:07.667811
